In [1]:
import requests

In [8]:
import pandas as pd

In [4]:
otp_payload = {
    'bld': 'MKD/01/0110/01100305/mkd01100305_01',
    'name': 'form'
}
header = {
            'Referer': 'http://marketdata.krx.co.kr/mdi',
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko Chrome/79.0.3945.130 Safari/537.36',
            'X-Requested-With': 'XMLHttpRequest'
}
otp_url = 'http://marketdata.krx.co.kr/contents/COM/GenerateOTP.jspx'

otp = requests.post('http://marketdata.krx.co.kr/contents/COM/GenerateOTP.jspx', data=otp_payload,
                         headers=header).text


payload = {
    'search_bas_yy': 2020,
    'gridTp': 'KRX',
    'pagePath': '/contents/MKD/01/0110/01100305/MKD01100305.jsp',
    'code': otp
}


In [5]:
req = requests.post('http://marketdata.krx.co.kr/contents/MKD/99/MKD99000001.jspx', data=payload, headers=header).json()

In [15]:
base = []
for year in range(2009, 2021):
    payload['search_bas_yy'] = year
    req = requests.post('http://marketdata.krx.co.kr/contents/MKD/99/MKD99000001.jspx', data=payload, headers=header).json()['block1']
    base += req

In [17]:
holiday = pd.DataFrame(base)

In [18]:
holiday['calnd_dd'] = pd.to_datetime(holiday['calnd_dd'])
holiday1 = holiday[['calnd_dd', 'holdy_nm', 'kr_dy_tp']].copy()

In [19]:
holiday1

,calnd_dd,holdy_nm,kr_dy_tp
0,2009-01-01,,목요일
1,2009-01-26,,월요일
2,2009-01-27,,화요일
3,2009-05-01,,금요일
4,2009-05-05,,화요일
...,...,...,...
158,2020-10-01,추석,목요일
159,2020-10-02,추석,금요일
160,2020-10-09,한글날,금요일
161,2020-12-25,성탄절,금요일


In [21]:
holiday1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 163 entries, 0 to 162
Data columns (total 3 columns):
calnd_dd    163 non-null datetime64[ns]
holdy_nm    163 non-null object
kr_dy_tp    163 non-null object
dtypes: datetime64[ns](1), object(2)
memory usage: 3.9+ KB


In [23]:
holiday1.set_index('calnd_dd',inplace=True)

In [24]:
holiday1

,holdy_nm,kr_dy_tp
calnd_dd,,
2009-01-01,,목요일
2009-01-26,,월요일
2009-01-27,,화요일
2009-05-01,,금요일
2009-05-05,,화요일
...,...,...
2020-10-01,추석,목요일
2020-10-02,추석,금요일
2020-10-09,한글날,금요일


In [25]:
holiday1.to_pickle('../Database/krxholidays.pickle')

In [1]:
import pandas as pd

In [2]:
holiday = pd.read_pickle('../Database/krxholidays.pickle')

In [15]:
bizday = pd.bdate_range(start='20090101', end='20201231')

In [16]:
krxbiz = list(set(bizday) - set(holiday.index))

In [17]:
krxbiz.sort()

In [18]:
krxbiz

[Timestamp('2009-01-02 00:00:00', freq='B'),
 Timestamp('2009-01-05 00:00:00', freq='B'),
 Timestamp('2009-01-06 00:00:00', freq='B'),
 Timestamp('2009-01-07 00:00:00', freq='B'),
 Timestamp('2009-01-08 00:00:00', freq='B'),
 Timestamp('2009-01-09 00:00:00', freq='B'),
 Timestamp('2009-01-12 00:00:00', freq='B'),
 Timestamp('2009-01-13 00:00:00', freq='B'),
 Timestamp('2009-01-14 00:00:00', freq='B'),
 Timestamp('2009-01-15 00:00:00', freq='B'),
 Timestamp('2009-01-16 00:00:00', freq='B'),
 Timestamp('2009-01-19 00:00:00', freq='B'),
 Timestamp('2009-01-20 00:00:00', freq='B'),
 Timestamp('2009-01-21 00:00:00', freq='B'),
 Timestamp('2009-01-22 00:00:00', freq='B'),
 Timestamp('2009-01-23 00:00:00', freq='B'),
 Timestamp('2009-01-28 00:00:00', freq='B'),
 Timestamp('2009-01-29 00:00:00', freq='B'),
 Timestamp('2009-01-30 00:00:00', freq='B'),
 Timestamp('2009-02-02 00:00:00', freq='B'),
 Timestamp('2009-02-03 00:00:00', freq='B'),
 Timestamp('2009-02-04 00:00:00', freq='B'),
 Timestamp

In [20]:
krxbiz = pd.to_datetime(krxbiz)

In [22]:
x = pd.DataFrame(krxbiz, columns = ['krxbiz'])
x

,krxbiz
0,2009-01-02
1,2009-01-05
2,2009-01-06
3,2009-01-07
4,2009-01-08
...,...
2963,2020-12-23
2964,2020-12-24
2965,2020-12-28
2966,2020-12-29


In [23]:
x.to_pickle('../Database/krxbiz.pickle')

In [31]:
pd.to_datetime('20100101') < x['krxbiz'] 

0       False
1       False
2       False
3       False
4       False
        ...  
2963     True
2964     True
2965     True
2966     True
2967     True
Name: krxbiz, Length: 2968, dtype: bool

In [25]:
x1 = x.copy()
x1.index = x1['krxbiz']
x1

,krxbiz
krxbiz,
2009-01-02,2009-01-02
2009-01-05,2009-01-05
2009-01-06,2009-01-06
2009-01-07,2009-01-07
2009-01-08,2009-01-08
...,...
2020-12-23,2020-12-23
2020-12-24,2020-12-24
2020-12-28,2020-12-28


In [40]:
x2 = x1.rename(index = {'krxbiz':'date'})
x2

,krxbiz
krxbiz,
2009-01-02,2009-01-02
2009-01-05,2009-01-05
2009-01-06,2009-01-06
2009-01-07,2009-01-07
2009-01-08,2009-01-08
...,...
2020-12-23,2020-12-23
2020-12-24,2020-12-24
2020-12-28,2020-12-28


In [41]:
x1.index.name = 'date'

In [43]:
x1.to_pickle('../Database/krxbiz.pickle')

In [35]:
a = x1['krxbiz'].loc[pd.to_datetime('20200101'):pd.to_datetime('20200131')]

In [36]:
pd.DataFrame(index = a)

""
krxbiz
2020-01-02
2020-01-03
2020-01-06
2020-01-07
2020-01-08
2020-01-09
2020-01-10
2020-01-13
2020-01-14
